# Setup
Before we proceed, we need to... 
- import any necessary libraries
- create a function to help display API responses
- and fill out our config object

In [ ]:
import requests
import json
import random

def view(r: dict) -> None:
    if r.status_code == 400:
        raise RuntimeError(f"something went wrong - {r.status_code}: {r.text}")
    elif r.status_code == 403:
        raise RuntimeError(f"something went wrong - {r.status_code}: {r.text}")
    print(json.dumps(r.json(), indent=4))

### Fill in the `config` object below with information provided to you by LightningDocs

This information will change between testing the API and using the API in actual production.

- `version` is the LightningDocs API version you wish to use for the translator.
- `keyID` is the Knackly API key provided to you.
- `tenancy` should be the name of the Knackly workspace.
- `secret` is the very long Knackly secret key provided to you.
- `catalog_name` is the name of the catalog in which records and finished documents are stored. While testing and demonstrating this API you will use a catalog shared by other testers. As such, DO NOT SEND CONFIDENTIAL INFORMATION WHILE TESTING THE API. In actual production use, each client will access a catalog unique to them.
- `app_name` is the name of the application that runs the data you send and produces the appropriate documents.
- `refresh_token` is the token used to validate the external user you are presenting yourself as (more on that below).

Afterward, make sure to run the cell below. This ensures that all the variables are properly stored in memory.

In [ ]:
from util import LightningDocs, Knackly, Config

config = Config(
    LightningDocs(version="VERSION_HERE"),
    Knackly(
        tenancy="TENANCY_HERE",
        keyID="YOUR_KEYID_HERE",
        secret="YOUR_SECRET_HERE",
        catalog_name="CATALOG_NAME_HERE",
        app_name="APP_NAME_HERE",
        refresh_token="YOUR_REFRESH_TOKEN_HERE"
    ),
)

base_url = f"https://api.knackly.io/{config.knackly.tenancy}/api/v1"

# Interacting with the LightningDocs API
- _Schema documentation can be found [here](https://github.com/LightningDocs/LightningDocs-Knackly-API/blob/master/LightningDocs_Schema.json)_
- _Postman documentation for the LightningDocs API can be found [here](https://documenter.getpostman.com/view/29599365/2s9YRGwoCT)_

The JSON Schema is provided so that we can validate our data file for completeness. With that information we can decide whether we want to produce a completed document or provide a pre-filled interview to the user. No example is given here because completeness is not necessary for the LightningDocs API to translate the data provided to it; it will translate any information that can be used to fill the interview and ignore any irrelevant information. 

### Sending JSON and getting back a valid Knackly JSON object<a id='ld_api'></a>

The following cell sends the contents of `valid.json` which follows the provided schema with information for a completely filled out loan.

In [ ]:
url = "https://api.lightningdocs.com/api/JsonConversion/Convert"

payload = {
    'version': config.lightningdocs.version
}

with open("json/lightningdocs/valid.json", "r") as infile:
    data = json.load(infile)
    
response = requests.post(url, params=payload, json=data)

view(response)

The following cell uses data from `incomplete.json`. This is valid JSON that follows the schema, but is largely incomplete.

In [ ]:
with open("json/lightningdocs/incomplete.json", "r") as infile:
    data = json.load(infile)
    
response = requests.post(url, params=payload, json=data)

view(response)

### Sending irrelevant / invalid JSON...

#### Sending data that is irrelevant
Irrelevant data will simply be ignored by the API. Only relevant information will be converted.

In [ ]:
with open("json/lightningdocs/irrelevant.json", "r") as infile:
    data = json.load(infile)
    
response = requests.post(url, params=payload, json=data)

view(response)

#### Sending data that does not follow the schema
Validating your data against the schema is *your* responsibility. The API does not check the sent data against the schema, so any values that are not in the format Knackly expects **will not** be corrected.

For example, in the following cell file `invalid.json` is used. According to the schema, the `closingDate` key should follow the `date` format *("YYYY-MM-DD")*. However, we are sending a value of "January 14th, 2001".

In [ ]:
with open("json/lightningdocs/invalid.json", "r") as infile:
    data = json.load(infile)
    
response = requests.post(url, params=payload, json=data)

view(response)

# Interacting with the Knackly API
The full Knackly documentation is available on Postman [here](https://documenter.getpostman.com/view/6868588/SzS7QReU#intro). Note that this is maintained by Knackly themselves, and not associated directly with LightningDocs.

Nearly all requests to the Knackly API must contain an access token provided in the request header. In order to obtain this access token, we will need the `keyID` and `secret` provided to us by LightningDocs.

## Access Token<a id='access_token'></a>

In [ ]:
url = f"https://api.knackly.io/{config.knackly.tenancy}/api/v1/auth/login"

payload = {"KeyID": config.knackly.keyID, "Secret": config.knackly.secret}

response = requests.post(url, json=payload)
access_token = response.json()["token"]

view(response)

The access token needs to be included in the header of each subsequent response, so we will create the `headers` object which will be included in nearly all future requests.

In [ ]:
headers = {"Authorization": f"Bearer {access_token}"}

## External Users
Because the Knackly application ties all actions that create and modify a record to a specific user (even when those actions are taken through the API), we must identify ourself as an external user using the provided `refresh token`. This is analagous to a recovery key for the external user: the `refresh token` itself isn't the "password" to the external token, but possession of it allows generation of the `user token`, which _is_ used to uniquely identify an external user.

### Refreshing an external user's `Token`<a id='refreshing_token'></a>
The `user token` for each external user will expire after a certain period of time. If the user token is found to be invalid, we can use the provided refresh token to generate a new, valid token for the associated external user.

In [ ]:
url = f"{base_url}/externals/refresh"

payload = {"Refresh_Token": config.knackly.refresh_token}

response = requests.post(url, headers=headers, json=payload)
user_token = response.json()["Token"]

view(response)

### Validating an external user
It is a good idea to validate that the external user's token is valid before attempting to make other API calls involving that user.

- If valid, the body of the GET response will be empty.
- If invalid, the body of the GET response will contain an error message.

In [ ]:
# Valid user
url = f"{base_url}/externals/valid/{user_token}"

response = requests.get(url, headers=headers)
response.text

In [ ]:
# Invalid user example
invalid_user_token = "obviouslyIncorrectToken123"
url = f"{base_url}/externals/valid/{invalid_user_token}"

response=requests.get(url, headers=headers)
response.text

### Creating a new record<a id='new_record'></a>
Note that in this example, we are sending information (`payload`) along with the API request, in order to have parts of the interview already filled out. This step is optional, and we could opt not to send any information, resulting in a completely blank interview if we would like our users to start from scratch.

In [ ]:
url = f"{base_url}/catalogs/{config.knackly.catalog_name}/apps/{config.knackly.app_name}"

params = {
    "external": user_token
}

payload = {
    "lender": {
        "id$": "1",
        "name": "Larry the Lender",
        "address": {
            "id$": "2",
            "city": "Lafayette",
            "street": "1234 Laniakae Lane",
            "zip": "98765",
            "state": "California"
        }
    }
}

response = requests.post(url, params=params, json=payload, headers=headers)

view(response)

Now that the record has been created, we will use the `url` returned to us as part of the `apps` array in the API response to build a new url which can be given to the external user, allowing the user to open up the interview and edit this record in their browser. Alternatively, our application can use the URL to present the interview to the user within a frame.

Note the `return_complete` and `return_incomplete` parameters in the cell below. These are optional, but will redirect the user's browser to the associated url when they hit either "Complete" or "Finish Later" from the interview screen, respectively. In this example we are redirecting to a youtube video, but in a real application this url could itself be an API call that would alert our system that the user has finished their work, at which point our application can take further action.<a id='construct_url'></a>

In [ ]:
apps_for_record = response.json()["apps"] # This is an array of app objects. Since our catalog only has one app, our array has a length of 1
app_url = apps_for_record[0]["url"]

from urllib.parse import urlencode
params = {
    'external': user_token,
    "return_complete": "https://www.youtube.com/watch?v=dQw4w9WgXcQ",
    "return_incomplete": "https://example.com/"
}
external_url = f"{app_url}?{urlencode(params)}"

print(external_url)

### Updating an Existing Record<a id='update_record'></a>
The Knackly API also allows for modifying existing records. In order to do this, we must know the specific `id` of a record to perform the modification on. For demonstration purposes, in the example below we are just retrieving the first `id` returned from an API call that lists all records in a catalog. 

*If following along with each cell, this should be the same record that you created in the step [Creating a New Record](#new_record).*

---

The following is copied verbatim from the Knackly documentation (as of 2024-02-27):

*"**Important note:** This endpoint should be considered BETA. Its behavior may be refined over time. In particular, it may allow modifications that are later rejected as errors, and it may reject modifications that could later be allowed. This is because the rules for exactly when and how records are locked and unlocked are not 100% solid yet. So, expect minor adjustments moving forward."*

In [ ]:
# Get a specific record_id
url = f"{base_url}/catalogs/{config.knackly.catalog_name}/items"
response = requests.get(url, headers=headers)

record_id = response.json()[0]["id"]
print(record_id)

Once we have the `id` of the record we wish to modify, we simply make a PUT request with our new data being sent in the body of the call.

Note that the data sent in the body of our request will **completely replace** all of the data currently stored in the record. If we wanted to modify or add to what currently exists in the record, we would have to [get the details for a specific record](#get_record_details), modify the data, and then send this PUT request with the newly modified data.

In [ ]:
url = f"{base_url}/catalogs/{config.knackly.catalog_name}/items/{record_id}"

payload = {
    "lender": {
        "id$": "1",
        "name": "Manny the Money Giver",
        "address": {
            "id$": "2",
            "city": "Missoula",
            "street": "1234 Madeline Ave",
            "zip": "59801",
            "state": "Montana"
        }
    }
}

response = requests.put(url, json=payload, headers=headers)

view(response)

### Checking app status and getting record details
After a user completes the interview, it will take the application a few moments (usually on the order of 20 seconds) to assemble the documents before we can download them. Moreover, if the user chose to end the interview by pressing "Finish Later" instead of "Complete", checking the status of the app for the record will allow us to see whether it is complete and documents are available.

#### GET - Get status of an app on a record<a id='get_status'></a>
Part of the response in the API call to create a record is the record's `id`. This is used to reference that specific record, and will be useful in future calls.

In [ ]:
record_id = response.json()["id"]

url = f"{base_url}/catalogs/{config.knackly.catalog_name}/items/{record_id}/apps/{config.knackly.app_name}"

response = requests.get(url, headers=headers)

view(response)

#### GET - Get record details<a id='retrieve_data'></a>
Checking the status of an app will return information including URLs to the downloadable documents, but we can retrieve more detailed information about the record if necessary.

In [ ]:
url = f"{base_url}/catalogs/{config.knackly.catalog_name}/items/{record_id}"

response = requests.get(url, headers=headers)

view(response)

As seen above, part of the response from this API call is the `files` array. Each element in this array is an object containing:
- `name` the actual filename of the downloadable document.
- `url` the permanent url to the downloadable documents. Can only be accessed via an authorized api key.
- `publicURL` the time-sensitive (5 minutes) link to the downloadble document.

### Webhooks
Knackly supports webhooks, meaning we can receive POST requests _from_ Knackly to our own program, and then do whatever we want with the provided data. As of now, webhooks will only fire on the `catalog.app.completed` event, which is triggered whenever any app in a given catalog is completed.

The data that is included with each event is as follows:

| property | description |
| --- | --- |
| eventId	| a unique ID identifying this event (not currently used for much) |
| eventName	| the name of the event -- currently always "catalog.app.completed" |
| catalog	| the name of the Knackly catalog in which the event occurred |
| record	| the ID of the record in the above catalog, on which an app was completed. Use this to retrieve data or documents. |
| app	| the name of the app that was completed |
| docsStarted	| the date/time stamp when the user completed the app (documents started generating) |
| docsFinished	| the date/time stamp when documents finished generating for the app |
| docCount	| the number of documents generated by the app |
| userType	| either "regular", "external", or "api" |
| userName	| the user's name (for regular or external users) or the API key name (if app was run directly from the API) |
| userEmail	| the user's email address (for regular and external users only) |

#### Get webhooks
Optionally, an `id` can be passed to the url to get information a specific webhook

In [ ]:
url = f"{base_url}/webhooks"

response = requests.get(url, headers=headers)

view(response)

#### Register a webhook

In [ ]:
url = f"{base_url}/webhooks"

data = {
    "url": "YOUR_URL_HERE",
    "events": [
        "catalog.app.completed"
    ],
    "catalogs": [
        config.knackly.catalog_name
    ]
}

response = requests.post(url, headers=headers, json=data)

response.text

#### Unregister a webhook

In [ ]:
url = f"{base_url}/webhooks/YOUR_WEBHOOK_ID_HERE"

response = requests.delete(url, headers=headers)

response.text

## Records
Knackly provides an easy way to retrieve all records from a given catalog.

### Get all records in a catalog

In [ ]:
url = f"{base_url}/catalogs/{config.knackly.catalog_name}/items"

payload = {
    "status": "Ok", # Optional parameter to filter results to only contain records with a particular status
}

response = requests.get(url, headers=headers, params=payload)

view(response)

The above API call only returns **metadata** about the records in a catalog. In order to get more specific details for a given record, we must do that through an individual API call.

### Get details for a specific record<a id='get_record_details'></a>

In [ ]:
record_id = random.choice(response.json())["id"] # In practice, this would be a specific id. A random choice of the valid id's is made here for demonstration purposes

url = f"{base_url}/catalogs/{config.knackly.catalog_name}/items/{record_id}"

new_response = requests.get(url, headers=headers)

view(new_response)

The structure of the response is split into three sections:
- `metadata` information about the record itself, such as a unique id, status, and last modified date
- `data` the actual data / answers given in the record
- `apps` any apps that pertain to this record, including URLs to any downloadable documents produced by the apps

# Workflow Examples

### Produce documents for a complete data set

1. [Call the LightningDocs API](#ld_api) to convert the data.
2. [Create an `access token`.](#access_token)
3. [Refresh our external user](#refreshing_token)'s token (if it is not currently valid).
4. [Create a new record](#new_record) with the data returned from the LD API as the `payload`.
5. After waiting sufficient time for Knackly to finish running the application, [get the status of the app](#get_status) on the record.
6. Use the document URLs provided in the status to download the completed documents.

### Create a partially filled interview for a user

1. [Call the LightningDocs API](#ld_api) to convert the data.
2. [Create an `access token`](#access_token) (if we haven't done so recently).
3. [Refresh our external user](#refreshing_token)'s `token` (if it is not currently valid).
4. [Create a new record](#new_record) with the data returned from the LD API as the `payload`.
5. Using the URL returned in the API response to creating the record, [construct a URL](#construct_url) to give to the user (or to present the interview to them).
6. Upon receiving notification that the user has completed the interview and—after waiting sufficient time for Knackly to finish running the application—[get the status of the app](#get_status) on the record.
7. Use the document URLs provided in the status to download the completed documents.
8. (Optional) [Retrieve the data](#retrieve_data) (now flush with the user's answers to the interview questions) from the record. (If we want to import the data into our application.)

### Modify a record after it has been filled out

1. [Call the LightningDocs API](#ld_api) to convert the data.
2. [Create an `access token`](#access_token) (if we haven't done so recently).
3. [Refresh our external user](#refreshing_token)'s `token` (if it is not currently valid).
4. [Retrieve the current data](#get---get-record-details) stored in the record to be updated.
5. Modify the entire JSON object.
6. [Update the existing record](#updating-an-existing-record) using the new data.